In [ ]:
!pip install albumentations==0.4.6


     |████████████████████████████████| 117 kB 5.1 MB/s 
     |████████████████████████████████| 948 kB 32.1 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=ac35964e11084e482be2142576ad900aa3f07e084dfc8e03299925d2e4f1aad5
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
#!git clone  https://github.com/mrzhu-cool/pix2pix-pytorch.git

Cloning into 'pix2pix-pytorch'...
remote: Enumerating objects: 68, done.
remote: Total 68 (delta 0), reused 0 (delta 0), pack-reused 68
Unpacking objects: 100% (68/68), done.


In [ ]:
#!unzip /content/pix2pix-pytorch/dataset/facades.zip

Archive:  /content/pix2pix-pytorch/dataset/facades.zip
   creating: facades/
   creating: facades/test/
   creating: facades/test/a/
  inflating: facades/test/a/cmp_b0001.png  
  inflating: facades/test/a/cmp_b0002.png  
  inflating: facades/test/a/cmp_b0003.png  
  inflating: facades/test/a/cmp_b0004.png  
  inflating: facades/test/a/cmp_b0013.png  
  inflating: facades/test/a/cmp_b0023.png  
  inflating: facades/test/a/cmp_b0026.png  
  inflating: facades/test/a/cmp_b0028.png  
  inflating: facades/test/a/cmp_b0034.png  
  inflating: facades/test/a/cmp_b0035.png  
  inflating: facades/test/a/cmp_b0047.png  
  inflating: facades/test/a/cmp_b0049.png  
  inflating: facades/test/a/cmp_b0054.png  
  inflating: facades/test/a/cmp_b0055.png  
  inflating: facades/test/a/cmp_b0057.png  
  inflating: facades/test/a/cmp_b0061.png  
  inflating: facades/test/a/cmp_b0068.png  
  inflating: facades/test/a/cmp_b0071.png  
  inflating: facades/test/a/cmp_b0073.png  
  inflating: facades/test/a/cmp

In [ ]:
#!cp -r /content/facades /content/drive/MyDrive/GANs

# Used Dataset : https://drive.google.com/drive/folders/1Zvt38htE48cjp53P3O-AeVndE9nJqkpi?usp=sharing 

In [ ]:
import albumentations 
from albumentations.pytorch import ToTensorV2

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


 #DEVICE= "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "/content/drive/MyDrive/GANs/facades/train"
VAL_DIR = "/content/drive/MyDrive/GANs/facades/test"
LEARNING_RATE = 2e-4
BATCH_SIZE = 16
NUM_WORKERS = 2
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
LAMBDA_GP = 10
NUM_EPOCHS = 200
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"


In [ ]:
import numpy as np
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision.utils import save_image
import torchvision.transforms as transforms

class MapDataset(Dataset): #should pass dir of dataset to be trained and tested which it contains 2 files(a & b)
  def __init__(self, root_dir):
    self.root_dir = root_dir
    self.root_dir_a = os.path.join(self.root_dir, "a")
    self.root_dir_b = os.path.join(self.root_dir, "b")
    
    self.list_files_a = os.listdir(self.root_dir_a )
    self.list_files_b = os.listdir(self.root_dir_b)

  def __len__(self):
    #print("HI")
    #print(len(self.list_files_a))
    #print(len(self.list_files_b))
    return len(self.list_files_b)

  def __getitem__(self, index):
    input_img_file = self.list_files_a[index]
    target_img_file = self.list_files_b[index]

    #print("input_img_file" ,input_img_file)
    #print("target_img_file" ,target_img_file)

    input_img_path = os.path.join(self.root_dir_a, input_img_file)
    target_img_path = os.path.join(self.root_dir_b, target_img_file)

    #print ("input_img_path"  ,input_img_path)
    #print ("target_img_path"  ,target_img_path)
    
    input_image = np.array(Image.open(input_img_path))
    target_image = np.array(Image.open(target_img_path))

    both_transform = A.Compose(
    [A.Resize(width=256, height=256),], additional_targets={"image0": "image"},
)
    transform_only_input = A.Compose(
    [
    A.HorizontalFlip(p=0.5),A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
    )

    transform_only_mask = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
  )
    augmentations = both_transform(image=input_image, image0=target_image)
    input_image = augmentations["image"]
    target_image = augmentations["image0"]
    input_image = transform_only_input(image=input_image)["image"]
    target_image = transform_only_mask(image=target_image)["image"]

    return input_image, target_image

'''
if __name__ == "__main__":
    dataset = MapDataset("/content/drive/MyDrive/GANs/facades/train")
    loader = DataLoader(dataset, batch_size=5)
    for x, y in loader:
        print(x.shape)
'''

'\nif __name__ == "__main__":\n    dataset = MapDataset("/content/drive/MyDrive/GANs/facades/train")\n    loader = DataLoader(dataset, batch_size=5)\n    for x, y in loader:\n        print(x.shape)\n'

# Discriminator

In [ ]:
import torch
import torch.nn as nn


class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(CNNBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, 4, stride, 1, bias=False, padding_mode="reflect"
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels * 2,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                CNNBlock(in_channels, feature, stride=1 if feature == features[-1] else 2),
            )
            in_channels = feature

        layers.append(
            nn.Conv2d(
                in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"
            ),
        )

        self.model = nn.Sequential(*layers)

    def forward(self, x, y):
        x = torch.cat([x, y], dim=1)
        x = self.initial(x)
        x = self.model(x)
        return x


def test():
    x = torch.randn((1, 3, 256, 256))
    y = torch.randn((1, 3, 256, 256))
    model = Discriminator(in_channels=3)
    preds = model(x, y)
    print(model)
    print(preds.shape)

'''
if __name__ == "__main__":
    test()'''

'\nif __name__ == "__main__":\n    test()'

# Generator

In [ ]:
import torch
import torch.nn as nn


class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )

        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x


class Generator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )
        self.down1 = Block(features, features * 2, down=True, act="leaky", use_dropout=False)
        self.down2 = Block(
            features * 2, features * 4, down=True, act="leaky", use_dropout=False
        )
        self.down3 = Block(
            features * 4, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down4 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down5 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.down6 = Block(
            features * 8, features * 8, down=True, act="leaky", use_dropout=False
        )
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features * 8, features * 8, 4, 2, 1), nn.ReLU()
        )

        self.up1 = Block(features * 8, features * 8, down=False, act="relu", use_dropout=True)
        self.up2 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up3 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True
        )
        self.up4 = Block(
            features * 8 * 2, features * 8, down=False, act="relu", use_dropout=False
        )
        self.up5 = Block(
            features * 8 * 2, features * 4, down=False, act="relu", use_dropout=False
        )
        self.up6 = Block(
            features * 4 * 2, features * 2, down=False, act="relu", use_dropout=False
        )
        self.up7 = Block(features * 2 * 2, features, down=False, act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features * 2, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d7], 1))
        up3 = self.up3(torch.cat([up2, d6], 1))
        up4 = self.up4(torch.cat([up3, d5], 1))
        up5 = self.up5(torch.cat([up4, d4], 1))
        up6 = self.up6(torch.cat([up5, d3], 1))
        up7 = self.up7(torch.cat([up6, d2], 1))
        return self.final_up(torch.cat([up7, d1], 1))


def test():
    x = torch.randn((1, 3, 256, 256))
    model = Generator(in_channels=3, features=64)
    preds = model(x)
    print(preds.shape)

'''
if __name__ == "__main__":
    test()'''

'\nif __name__ == "__main__":\n    test()'

In [ ]:
import torch
from torchvision.utils import save_image

def save_some_examples(gen, val_loader, epoch, folder):
    x, y = next(iter(val_loader))
    x, y = x.to(DEVICE), y.to(DEVICE)

    gen.eval()
    with torch.no_grad():
        y_fake = gen(x)
        y_fake = y_fake * 0.5 + 0.5  # remove normalization#
        save_image(y_fake, folder + f"/y_gen_{epoch}.png")
        save_image(x * 0.5 + 0.5, folder + f"/input_{epoch}.png")
        if epoch == 1:
            save_image(y * 0.5 + 0.5, folder + f"/label_{epoch}.png")
    gen.train()


def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


# Train

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision.utils import save_image

torch.backends.cudnn.benchmark = True

def train_fn(disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler,):
    
    loop = tqdm(loader, leave=True)

    for idx, (x, y) in enumerate(loop):
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        #x=x.cuda()
        #y=y.cuda()
        # Train Discriminator
        with torch.cuda.amp.autocast():
            y_fake = gen(x)
            D_real = disc(x, y)
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake = disc(x, y_fake.detach())
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1_loss(y_fake, y) * L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 10 == 0:
            loop.set_postfix(
                D_real=torch.sigmoid(D_real).mean().item(),
                D_fake=torch.sigmoid(D_fake).mean().item(),
            )


def main():
    disc = Discriminator(in_channels=3).to(DEVICE)
    gen = Generator(in_channels=3, features=64).to(DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999),)
    opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    L1_LOSS = nn.L1Loss()

    if LOAD_MODEL:
        load_checkpoint(CHECKPOINT_GEN, gen, opt_gen, LEARNING_RATE, )
        load_checkpoint(CHECKPOINT_DISC, disc, opt_disc,LEARNING_RATE, )

    train_dataset = MapDataset(root_dir=TRAIN_DIR) ###################################3
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
    )
    

    
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()
    val_dataset = MapDataset(root_dir=VAL_DIR) ######################################################3
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    for epoch in range(NUM_EPOCHS):
        gen.cuda()
        disc.cuda()
        train_fn(
            disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler,
        )
        
        if SAVE_MODEL and epoch % 5 == 0:
            save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN)
            save_checkpoint(disc, opt_disc, filename=CHECKPOINT_DISC)
            torch.save(gen.state_dict(),'/content/saved_moedels/gen.h5' )
       
        save_some_examples(gen, val_loader, epoch, folder="evaluation2")


if __name__ == "__main__":
    main()

100%|██████████| 25/25 [00:06<00:00,  4.05it/s, D_fake=0.316, D_real=0.72]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 25/25 [00:06<00:00,  4.11it/s, D_fake=0.179, D_real=0.711]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 25/25 [00:05<00:00,  4.19it/s, D_fake=0.18, D_real=0.82]


...

100%|██████████| 25/25 [00:05<00:00,  4.19it/s, D_fake=0.162, D_real=0.776]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 25/25 [00:06<00:00,  4.16it/s, D_fake=0.261, D_real=0.768]


=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 25/25 [00:05<00:00,  4.22it/s, D_fake=0.569, D_real=0.4]


In [ ]:
!cp -r /content/saved_moedels/* /content/drive/MyDrive/GANs/Saved-Gans-weights

In [ ]:
!cp -r /content/evaluation2 /content/drive/MyDrive/GANs

In [ ]:
!cp -r /content/gen.pth.tar /content/drive/MyDrive/GANs